In [0]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F

In [0]:
data = [
 
  (11111 , 'CA', '2020-01-26'),
  (22222,  'CA', '2020-02-26'),
  (33333 , 'CA', '2020-03-26'),
  (44444 , 'CA', '2020-04-26'),
  (55555 , 'CA', '2020-05-26'),
  (66666 , 'CA', '2020-06-26'),
  (77777 , 'CA', '2020-07-26'),
  (88888 , 'CA', '2020-06-10')
  
  ]
 
columns = ['attom_id', 'state_code', 'sell_date']
base_df = spark.createDataFrame(data, columns)

In [0]:
from pyspark.sql.types import DateType

base_df = base_df.withColumn('sell_date', to_date(col("sell_date"))) \
        .withColumn("is_current", lit(True)) \
        .withColumn("start_date", to_date(col("sell_date"))) \
        .withColumn("end_date", lit(None).cast(DateType()))

In [0]:
base_df.display()

attom_id,state_code,sell_date,is_current,start_date,end_date
11111,CA,2020-01-26,true,2020-01-26,null
22222,CA,2020-02-26,true,2020-02-26,null
33333,CA,2020-03-26,true,2020-03-26,null
44444,CA,2020-04-26,true,2020-04-26,null
55555,CA,2020-05-26,true,2020-05-26,null
66666,CA,2020-06-26,true,2020-06-26,null
77777,CA,2020-07-26,true,2020-07-26,null
88888,CA,2020-06-10,true,2020-06-10,null


saved in workspace\default\demo_table_one

In [0]:
base_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("demo_table_one")

In [0]:
%sql
select * from demo_table_one

attom_id,state_code,sell_date,is_current,start_date,end_date
11111,CA,2020-01-26,true,2020-01-26,null
22222,CA,2020-02-26,true,2020-02-26,null
33333,CA,2020-03-26,true,2020-03-26,null
44444,CA,2020-04-26,true,2020-04-26,null
55555,CA,2020-05-26,true,2020-05-26,null
66666,CA,2020-06-26,true,2020-06-26,null
77777,CA,2020-07-26,true,2020-07-26,null
88888,CA,2020-06-10,true,2020-06-10,null


In [0]:
new_data = [
  (11111 , 'WW', '2020-02-26'),
  (88888 , 'WW', '2020-06-10'),
  (55555 , 'WW', '2020-03-15'),
  (99999,  'WW', '2020-07-01')
]

In [0]:
new_data = spark.createDataFrame(new_data, columns)
new_data = new_data.withColumn('sell_date', to_date(col("sell_date"))) 
       

In [0]:
from delta.tables import DeltaTable
deltaTable = DeltaTable.forName(spark, "demo_table_one") 

scd type-2

In [0]:
(deltaTable.alias("target").merge(new_data.alias("src"), 
                                 "src.attom_id = target.attom_id AND target.is_current = true")
                        .whenMatchedUpdate(
                            condition="src.state_code <> target.state_code AND target.is_current = true",
                            set = {
                                "is_current": lit(False),
                                "end_date": current_date()
                            }                      
                        )
                        .whenNotMatchedInsert(
                            values={
                                "attom_id": "src.attom_id",
                                "state_code": "src.state_code",
                                "sell_date": "src.sell_date",
                                "is_current": lit(True),
                                "start_date": current_date(),
                                "end_date": lit(None).cast("date")
                            }
                        )
                        .execute()
                        )

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("SELECT * FROM demo_table_one ORDER BY attom_id, start_date").display()


attom_id,state_code,sell_date,is_current,start_date,end_date
11111,CA,2020-01-26,false,2020-01-26,2025-09-23
11111,WW,2020-02-26,true,2025-09-23,null
22222,CA,2020-02-26,true,2020-02-26,null
33333,CA,2020-03-26,true,2020-03-26,null
44444,CA,2020-04-26,true,2020-04-26,null
55555,CA,2020-05-26,false,2020-05-26,2025-09-23
55555,WW,2020-03-15,true,2025-09-23,null
66666,CA,2020-06-26,true,2020-06-26,null
77777,CA,2020-07-26,true,2020-07-26,null
88888,CA,2020-06-10,false,2020-06-10,2025-09-23


without condition

In [0]:
(deltaTable.alias("target").merge(new_data.alias("src"), 
                                 "src.attom_id = target.attom_id AND target.is_current = true")
                        .whenMatchedUpdate(
                            set = {
                                "is_current": lit(False),
                                "end_date": current_date()
                            }                      
                        )
                        .whenNotMatchedInsert(
                            values={
                                "attom_id": "src.attom_id",
                                "state_code": "src.state_code",
                                "sell_date": "src.sell_date",
                                "is_current": lit(True),
                                "start_date": current_date(),
                                "end_date": lit(None).cast("date")
                            }
                        )
                        .execute()
                        )

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from demo_table_one order by attom_id, state_code

attom_id,state_code,sell_date,is_current,start_date,end_date
11111,CA,2020-01-26,false,2020-01-26,2025-09-23
22222,CA,2020-02-26,true,2020-02-26,null
33333,CA,2020-03-26,true,2020-03-26,null
44444,CA,2020-04-26,true,2020-04-26,null
55555,CA,2020-05-26,false,2020-05-26,2025-09-23
66666,CA,2020-06-26,true,2020-06-26,null
77777,CA,2020-07-26,true,2020-07-26,null
88888,CA,2020-06-10,false,2020-06-10,2025-09-23
99999,WW,2020-07-01,true,2025-09-23,null
